In [57]:
import pandapower as pp
import numpy as np
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net, vn_kv=220.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)

#create 220/110 kV transformer
pp.create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV")

#create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')

#create loads
pp.create_load(net, bus2, p_mw=60, controllable=False)
pp.create_load(net, bus3, p_mw=70, controllable=False)
pp.create_load(net, bus4, p_mw=10, controllable=False)

#create generators
eg = pp.create_ext_grid(net, bus1, min_p_mw=-1000, max_p_mw=1000)
g0 = pp.create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80,  vm_pu=1.01, controllable=True)
g1 = pp.create_gen(net, bus4, p_mw=100, min_p_mw=0, max_p_mw=100, vm_pu=1.01, controllable=True)

In [58]:
#creating cost function for generators and external grid
costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=10)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

In [59]:
pp.runopp(net, delta=1e-16)

In [60]:
#asigning individual cost to generators
net.poly_cost.cp1_eur_per_mw.at[costeg] = 10
net.poly_cost.cp1_eur_per_mw.at[costgen1] = 15
net.poly_cost.cp1_eur_per_mw.at[costgen2] = 12

In [61]:
pp.runopp(net, delta=1e-16)

In [62]:
#adding constraint to grid , Transformer loading max 50%
net.trafo["max_loading_percent"] = 50
pp.runopp(net, delta=1e-16)

In [63]:
#adding line constrain , line loading max 50%
net.line["max_loading_percent"] = 50
pp.runopp(net, delta=1e-16)

In [64]:
#adding limit to voltage 1.0<v<1.02(pu)
net.bus["min_vm_pu"] = 1.0
net.bus["max_vm_pu"] = 1.02
pp.runopp(net, delta=1e-16)

In [65]:
#LMP coding
tot_bus=4
d = [10 , 15, 12]   #cost of ext grid and generator
for j in range(tot_bus):
    b=[]
    c=[]
    pp.runopp(net, delta=1e-16)
    b.append(net.res_ext_grid.p_mw.at[0])   #adding the reult for external grid - p_mw to b[0]
    b.append(net.res_gen.p_mw.at[0])        #adding the reult for generator index[0] - p_mw to b[1]
    b.append(net.res_gen.p_mw.at[1])        #adding the reult for generator index[1] - p_mw to b[2]
    pp.create_load(net, net.bus.index[j], p_mw=1, controllable=False)    #adding additional 1 MW load to jth(0-4) bus to calulate LMP
    pp.runopp(net, delta=1e-16,numba=False)  #again running powerflow to get new results
    c.append(net.res_ext_grid.p_mw.at[0])    #adding the reult for external grid - p_mw to b[0]
    c.append(net.res_gen.p_mw.at[0])         #adding the reult for generator index[0] - p_mw to b[1]
    c.append(net.res_gen.p_mw.at[1])         #adding the reult for generator index[1] - p_mw to b[2]
    #print(b)
    #print(c)
    a=[]
    for i in range(len(d)):
        y=c[i]-b[i]                         #getting the differnce in both reults
        a.append(y)
    #print(a)
    cost=0
    for i in range(len(d)):
        f=a[i]*d[i]                        #multiply the cost to the difference to calculate LMP
        cost = cost + f
    print("The LMP cost of Bus", j+1 ," after adding 1 MW load is:", cost)


[49.90683880129212, 10.72183179502494, 81.86346210146748]
[50.906838837614046, 10.721831635840147, 81.86346222951283]
[1.0000000363219286, -1.5918479334686708e-07, 1.280453574281637e-07]
The LMP cost of Bus 1  after adding 1 MW load is: 9.999999511991675
[50.906838837614046, 10.721831635840147, 81.86346222951283]
[50.918611058534296, 11.076142060464253, 82.5052852533279]
[0.011772220920249765, 0.35431042462410645, 0.6418230238150642]
The LMP cost of Bus 2  after adding 1 MW load is: 13.134254864344864
[50.918611058534296, 11.076142060464253, 82.5052852533279]
[50.918598362980106, 12.076196027201947, 82.50524230540015]
[-1.269555419014523e-05, 1.0000539667376938, -4.29479277528344e-05]
The LMP cost of Bus 3  after adding 1 MW load is: 15.00016717039047
[50.918598362980106, 12.076196027201947, 82.50524230540015]
[50.918615231823246, 12.076124560428728, 83.50529912172453]
[1.6868843140116496e-05, -7.146677321934192e-05, 1.0000568163243884]
The LMP cost of Bus 4  after adding 1 MW load is: